In [1]:
import os
os.chdir('/Users/layne/Desktop/brain_tumor_preprocessed')

In [2]:
mixed_file_paths = os.listdir('.')

In [3]:
mixed_file_paths.remove('.DS_Store')

In [4]:
len(mixed_file_paths)

80

In [5]:
from natsort import natsorted

In [6]:
mixed_file_paths = natsorted(mixed_file_paths)

---

In [7]:
import numpy as np

In [8]:
mixed_arr = []
for path in mixed_file_paths:
    if 'healthy' in path:
        arr = np.array([0,path])
    elif 'tumor' in path:
        arr = np.array([1,path])
    mixed_arr.append(arr)

In [9]:
import pandas as pd

/Users/layne/.pyenv/versions/3.7.6/envs/jupyterlab/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [10]:
df = pd.DataFrame(mixed_arr, columns=['status', 'path'])

In [11]:
df.shape

(80, 2)

In [12]:
df

,status,path
0,0,healthy_0.jpg
1,0,healthy_1.jpg
2,0,healthy_2.jpg
3,0,healthy_3.jpg
4,0,healthy_4.jpg
...,...,...
75,1,tumor_35.jpg
76,1,tumor_36.jpg
77,1,tumor_37.jpg
78,1,tumor_38.jpg


---

In [13]:
import os
os.chdir('/Users/layne/Desktop/aiqc')

In [ ]:
import aiqc
from aiqc import examples

In [ ]:
from importlib import reload; aiqc.delete_db(True); reload(aiqc); aiqc.create_db()

---

In [ ]:
dataset_tab = aiqc.Dataset.Tabular.from_pandas(dataframe=df, name='xray', dtype={'status':'int','path':'string'})

In [ ]:
dir_path = '/Users/layne/Desktop/brain_tumor_preprocessed'

In [ ]:
dataset_images = aiqc.Dataset.Image.from_folder(dir_path)

In [ ]:
label = dataset_tab.make_label(columns=['status'])

In [ ]:
featureset = dataset_images.make_featureset()

In [ ]:
splitset = featureset.make_splitset(label_id=label.id, size_test=0.30)

---

In [ ]:
import keras
from keras import metrics
from keras.models import Sequential
from keras.callbacks import History
from keras.callbacks import Callback
from keras.layers import Conv1D, Dense, Dropout
from keras.constraints import maxnorm
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Flatten

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
def function_model_build(**hyperparameters):
    model = Sequential()
    
    model.add(Conv1D(16*hyperparameters['neuron_multiply'], kernel_size=hyperparameters['kernel_size'], input_shape=(120,90), padding='same', activation='relu', kernel_initializer='he_normal'))
    model.add(MaxPooling1D(pool_size=hyperparameters['pool_size']))
    model.add(Dropout(hyperparameters['dropout']))
    
    model.add(Conv1D(32*hyperparameters['neuron_multiply'], kernel_size=hyperparameters['kernel_size'], padding='same', activation='relu', kernel_initializer='he_normal'))
    model.add(MaxPooling1D(pool_size=hyperparameters['pool_size']))
    model.add(Dropout(hyperparameters['dropout']))

    model.add(Flatten())
    model.add(Dense(hyperparameters['dense_neurons']*hyperparameters['neuron_multiply'], activation='relu'))
    if hyperparameters['include_2nd_dense'] == True:
        model.add(Dropout(0.2))
        model.add(Dense(hyperparameters['dense_neurons']*hyperparameters['neuron_multiply'], activation='relu'))

    model.add(Dense(1, activation='sigmoid'))

    opt = keras.optimizers.Adamax(hyperparameters['learning_rate'])
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
def function_model_train(model, samples_train, samples_evaluate, **hyperparameters):   
    
    class EarlyStopper(Callback):
        def __init__(self, monitor='val_accuracy', baseline=0.90):
            super(EarlyStopper, self).__init__()
            self.monitor = monitor
            self.baseline = baseline

        def on_epoch_end(self, epoch, logs=None):
            logs = logs or {}
            metric = logs.get(self.monitor)
            if metric is not None:
                if 'accuracy' in self.monitor:             
                    if metric >= self.baseline:
                        print('Epoch %d: Exceeded accuracy threshold. Stopped training early' % (epoch))
                        self.model.stop_training = True
                elif 'loss' in self.monitor:
                    if metric <= self.baseline:
                        print('Epoch %d: Crossed below loss threshold. Stopped training early' % (epoch))
                        self.model.stop_training = True
    
    model.fit(
        samples_train["features"]
        , samples_train["labels"]
        , validation_data = (
            samples_evaluate["features"]
            , samples_evaluate["labels"]
        )
        , verbose = 0
        , batch_size = hyperparameters['batch_size']
        , callbacks=[History(), EarlyStopper()]
        , epochs = hyperparameters['epoch_count']
    )

    return model

In [ ]:
algorithm = aiqc.Algorithm.make(
    library = "keras"
    , analysis_type = "classification_binary"
    , function_model_build = function_model_build
    , function_model_train = function_model_train
)

In [ ]:
hyperparameters = {
    "include_2nd_dense": [True]
    , "neuron_multiply": [4,5,6]
    , "epoch_count": [200]
    , "learning_rate": [0.005, 0.01]
    , "pool_size": [2]
    , "dropout": [0.4]
    , "batch_size": [8]
    , "kernel_size": [2]
    , "dense_neurons": [64]
}

hyperparamset = aiqc.Hyperparamset.from_algorithm(
    algorithm_id = algorithm.id
    , description = "experimenting with neuron count, layers, and epoch count"
    , hyperparameters = hyperparameters
)

In [ ]:
batch = aiqc.Batch.from_algorithm(
    algorithm_id = algorithm.id
    , splitset_id = splitset.id
    , hyperparamset_id = hyperparamset.id
    , foldset_id = None
    , preprocess_id = None
)

In [ ]:
batch.run_jobs()

In [ ]:
batch.get_statuses()

In [ ]:
batch.metrics_to_pandas()

In [ ]:
aiqc.Job.get_by_id(3).hyperparamcombo.hyperparameters

In [ ]:
batch.plot_performance(max_loss=15, min_metric_2=0.50)

In [ ]:
aiqc.Job.get_by_id().results[0].plot_learning_curve()

In [ ]:
aiqc.Job.get_by_id().results[0].plot_confusion_matrix()